In [1]:
from FileSystem import *
import os
home_dir = os.getcwd()
print(home_dir)
import datetime as dt
from datetime import *
curtz = datetime.now().astimezone().tzinfo
tform = '%Y-%m-%d %H-%M-%S%z'
import re
import json

D:\Users\feelus\Repos\hash_sync


In [2]:
def print_tree(obj):
    s = myjson_dumps(tree_dump(obj))
    # проверка на кол-во строк ...
    print(s)
def load_snapshot(path):
    tmp = clear_json_comment(myjson_load(path))
    return tree_load(tmp['root']),tree_load(tmp['errors'])
def dump_snapshot(root,errors,path):
    myjson_dump({
        'errors':tree_dump(errors),
        'root':tree_dump(root)
    },path)


In [3]:
newtime_s = None
snapshot_json = None
oldtime_s = None
snapshot_bak = None
root = None
oldroot = None
errors = None
olderrors = None
def last_diff_dir(prefix,prefix_,exclude_dirs,load_snapshot,dump_snapshot):
    global newtime_s
    global oldtime_s
    global root
    global oldroot
    global errors
    global olderrors
    os.chdir(prefix_)
    newtime = datetime.now(curtz)
    newtime_s = newtime.strftime(tform)
    snapshot_json = '.files/last_snapshot '+newtime_s+'.json'
    def find_date_file(prefix,postfix,ls):
        return [s[len(prefix):-len(postfix)] for s in ls if \
                  s.startswith(prefix) and s.endswith(postfix)]
    if '.files' in os.listdir('.'):
        ls = os.listdir('.files\\')
        oldtime_list = find_date_file('last_snapshot ','.bak',ls)
        assert len(oldtime_list)<=1
        if len(oldtime_list)==1:
            oldtime_s = oldtime_list[0]
            snapshot_bak = '.files/last_snapshot '+oldtime_s+'.bak'

            oldroot,olderrors = load_snapshot(snapshot_bak) # <----
            newtime_list = find_date_file('last_snapshot ','.json',ls)
            assert len(newtime_list)<=1
            if len(newtime_list)==1:
                print('fast recovery')
                # сканируем-пересчитываем на основе .json
                # затем удаляем его и сохраняем новый
                newtime_s = newtime_list[0]
                snapshot_json = '.files/last_snapshot '+newtime_s+'.json'

                root,errors = load_snapshot(snapshot_json) # <----
                #root,errors = scan(prefix_,exclude_dirs)
                #calc_hashes(root,errors,oldnewroot,prefix)

                #os.remove(oldsnapshot_json)
                #dump_snapshot(root,errors,snapshot_json) # ---->
            else:
                print('recovery')
                # сканируем-пересчитываем на основе .bak
                # затем сохраняем новый
                root,errors = scan(prefix_,exclude_dirs)
                calc_hashes(root,errors,oldroot,prefix)

                dump_snapshot(root,errors,snapshot_json) # ---->
        else:
            print('simple update')
            # сканируем-пересчитываем на основе json
            # затем переименовываем его в .bak и сохраняем новый
            oldtime_list = find_date_file('last_snapshot ','.json',ls)
            assert len(oldtime_list)==1
            oldtime_s = oldtime_list[0]
            snapshot_bak = '.files/last_snapshot '+oldtime_s+'.bak'

            oldroot,olderrors = load_snapshot('.files/last_snapshot '+oldtime_s+'.json') # <----
            root,errors = scan(prefix_,exclude_dirs)
            calc_hashes(root,errors,oldroot,prefix)

            os.rename('.files/last_snapshot '+oldtime_s+'.json',snapshot_bak)
            dump_snapshot(root,errors,snapshot_json) # ---->
    else:
        print('create all')
        # записываем пустое дерево в .bak (дата на минуту раньше текущей)
        # сканируем-пересчитываем на основе .bak
        # затем сохраняем новый
        os.mkdir('.files')
        oldtime_s = (newtime - timedelta(minutes=1)).strftime(tform)
        snapshot_bak = '.files/last_snapshot '+oldtime_s+'.bak'
        oldroot = {}
        root,errors = scan(prefix_,exclude_dirs)
        calc_hashes(root,errors,oldroot,prefix)

        dump_snapshot(oldroot,{},snapshot_bak) # ---->
        dump_snapshot(root,errors,snapshot_json)

    try:
        oldroot_d = oldroot#{k:v for k,v in oldroot.items() if k!='__scan_errors__'}
        root_d = root#{k:v for k,v in root.items() if k!='__scan_errors__'}
        modified,old,new,strict_old,strict_new,touched = path_diff(oldroot_d,root_d)
        assert set(old)&set(strict_old) == set()
        assert set(new)&set(strict_new) == set()
        moved,old_dirs,new_dirs,old,new = hash_diff(old,new)
        assert set(old)&set(strict_old) == set()
        assert set(new)&set(strict_new) == set()

        newoldroot = hash_back_patch(root_d,modified,moved,old_dirs,new_dirs,
                              {**old,**strict_old},{**new,**strict_new},touched)
        #first_diff(root_d,new_root_d)
        assert oldroot_d==newoldroot

        e_modified,e_old,e_new,e_strict_old,e_strict_new,e_touched = path_diff(olderrors,errors)
        newerrors = path_patch(olderrors,e_modified,
                               {**e_old,**e_strict_old},{**e_new,**e_strict_new}, e_touched)
        assert errors==newerrors
    except BaseException as e:
        #print('exception catched, writing "exception_dump.json"')
        #with open('.files/exception_dump.json','w') as file:
        #    json.dump({'olderrors':olderrors,'errors':errors,'oldroot':oldroot,'root':root},
        #          file)
        raise e
    patch_name = '.files/patch '+oldtime_s+' to '+newtime_s+'.json'
    myjson_dump({
        'errors':path_patch_dump(e_modified,e_old,e_new,e_strict_old,e_strict_new,e_touched),
        'root':hash_patch_dump(modified,moved,old_dirs,new_dirs,
                              {**old,**strict_old},{**new,**strict_new},touched)
    },patch_name)
    os.remove(snapshot_bak)

# D

In [4]:
exclude_dirs = {r'D:\Users\feelus\YandexDisk',r'D:\$RECYCLE.BIN'}
prefix = 'D:'
prefix_ = prefix+'\\'
last_diff_dir(prefix,prefix_,exclude_dirs,load_snapshot,dump_snapshot)

simple update
readed .files/last_snapshot 2019-11-21 11-01-25+0300.json
dict_keys(['errors', 'root'])


HTML(value='')

[WinError 5] Отказано в доступе: 'D:\\System Volume Information'
('D:\\Users\\feelus\\cyg-home\\.trash\\2018-03-19 13-27-51%-cygdrive%-c%-Users%-feelus%-Repos%-vybory%-regions%-adygei@action=show&root=1000001&tvd=100100084849067&vrn=100100084849062&region=1&global=1&sub_region=1&prver=0&pronetvd=null&vibid=100100084849067&type=227', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('D:\\Users\\feelus\\cyg-home\\.trash\\2018-03-19 19-50-07%-cygdrive%-c%-Users%-feelus%-Repos%-vybory%-adygei@action=show&amp;tvd=100100084849067&amp;vrn=100100084849062&amp;region=1&amp;global=1&amp;sub_region=1&amp;prver=0&amp;pronetvd=null&amp;vibid=2012000364363&amp;type=227', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('D:\\Users\\feelus\\cyg-home\\.trash\\2018-03-19 19-50-07%-cygdrive%-c%-Users%-feelus%-Repos%-vybory%-adygei@action=show&amp;tvd=100100084849067&amp;vrn=100100084849062&amp;region=1&amp;global=1&amp;sub_region=1&amp;prver=0&amp;pron

('D:\\Users\\feelus\\cyg-home\\wget-downloads\\www.maiclub.ru\\cgi-bin\\picturmai.pl@1_PARAPLAN_GAGRA_DRUZH007.jpg%3A2003_09_gagra%3A%20%D0%94%D1%80%D1%83%D0%B6%D0%B8%D0%BD%D0%B8%D0%BD%20%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B5%D0%B9,%20%D0%BF%D0%B0%D1%80%D0%B0%D0%BF%D0%BB%D0%B0%D0%BD%20%D1%80%D0%BE%D0%B7%D0%BE', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('D:\\Users\\feelus\\cyg-home\\wget-downloads\\www.maiclub.ru\\cgi-bin\\picturmai.pl@1_PARAPLAN_GAGRA_DRUZH009.jpg%3A2003_09_gagra%3A%20%D0%A1%D1%82%D0%B0%D1%80%D1%82%20%D1%82%D0%B0%D0%BD%D0%B4%D0%B5%D0%BC%D0%B0%20%D0%B2%20%D0%BE%D0%BA%D0%BD%D0%BE%20%D0%BC%D0%B5%D0%B6%D0%B4%D1%83%20%D0%BE%D0%B1%D0%BB%D0%B0%', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('D:\\Users\\feelus\\cyg-home\\wget-downloads\\www.maiclub.ru\\cgi-bin\\picturmai.pl@IMG_2834.jpg%3A2006baranov%3A%20%D1%82%D0%B0%D0%BA%D0%BE%D0%B9%20%D0%BF%D0%B8%D0%BF%D0%B8%D0%BB%D0%B0%D1%86%20%D0%BD%D0%B0%D1%85%D0%BE%D0%B4%D0

HTML(value='')

D:\Users\feelus\AppData\Local\Microsoft\Windows\UsrClass.dat
[Errno 13] Permission denied: 'D:\\Users\\feelus\\AppData\\Local\\Microsoft\\Windows\\UsrClass.dat'

D:\Users\feelus\AppData\Local\Microsoft\Windows\UsrClass.dat.LOG1
[Errno 13] Permission denied: 'D:\\Users\\feelus\\AppData\\Local\\Microsoft\\Windows\\UsrClass.dat.LOG1'

D:\Users\feelus\NTUSER.DAT
[Errno 13] Permission denied: 'D:\\Users\\feelus\\NTUSER.DAT'

D:\Users\feelus\ntuser.dat.LOG1
[Errno 13] Permission denied: 'D:\\Users\\feelus\\ntuser.dat.LOG1'

start writing
writed .files/last_snapshot 2019-11-21 12-21-25+0300.json
--- ITERATION ---
--- ITERATION ---
--- FINAL ITERATION ---
start writing
writed .files/patch 2019-11-21 11-01-25+0300 to 2019-11-21 12-21-25+0300.json


# C

In [5]:
exclude_dirs = {r'D:\$RECYCLE.BIN'}
prefix = 'C:'
prefix_ = prefix+'\\'
def en_load_snapshot(path):
    with open(path,'r') as file:
        tmp = clear_json_comment(json.load(file))
    return tree_load(tmp['root']),tree_load(tmp['errors'])
def en_dump_snapshot(root,errors,path):
    with open(path,'w') as file:
        json.dump({
            'errors':tree_dump(errors),
            'root':tree_dump(root)
        },file,indent='\t')

last_diff_dir(prefix,prefix_,exclude_dirs,en_load_snapshot,en_dump_snapshot)

simple update


HTML(value='')

('C:\\hiberfil.sys', PermissionError(13, 'Процесс не может получить доступ к файлу, так как этот файл занят другим процессом.', None, 32))
('C:\\pagefile.sys', PermissionError(13, 'Процесс не может получить доступ к файлу, так как этот файл занят другим процессом.', None, 32))
[WinError 5] Отказано в доступе: 'C:\\System Volume Information'
[WinError 5] Отказано в доступе: 'C:\\Windows\\CSC\\v2.0.6'
[WinError 5] Отказано в доступе: 'C:\\Windows\\System32\\LogFiles\\WMI\\RtBackup'


HTML(value='')

C:\Windows\System32\config\SOFTWARE
[Errno 13] Permission denied: 'C:\\Windows\\System32\\config\\SOFTWARE'

C:\Windows\System32\config\SOFTWARE.LOG1
[Errno 13] Permission denied: 'C:\\Windows\\System32\\config\\SOFTWARE.LOG1'

C:\Windows\System32\config\SYSTEM
[Errno 13] Permission denied: 'C:\\Windows\\System32\\config\\SYSTEM'

C:\Windows\System32\config\SYSTEM.LOG1
[Errno 13] Permission denied: 'C:\\Windows\\System32\\config\\SYSTEM.LOG1'

--- ITERATION ---
--- FINAL ITERATION ---
start writing
writed .files/patch 2019-11-21 11-37-35+0300 to 2019-11-21 12-24-38+0300.json


# YD

In [6]:
exclude_dirs = {}
prefix = r'D:\Users\feelus\YandexDisk'
prefix_ = prefix+'\\'
last_diff_dir(prefix,prefix_,exclude_dirs,load_snapshot,dump_snapshot)

simple update
readed .files/last_snapshot 2019-11-21 11-55-30+0300.json
dict_keys(['errors', 'root'])


HTML(value='')

('D:\\Users\\feelus\\YandexDisk\\_programming\\_lang_cpp\\поиск файлов и msdn _html\\msdn_html-куски\\z_windows development\\z_data acces and storage\\z_Local File Systems\\z_Directory Management\\About Directory Management (Windows)_files\\030c41d9079671d09a62d8e2c1db6973.gif', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('D:\\Users\\feelus\\YandexDisk\\_programming\\_lang_cpp\\поиск файлов и msdn _html\\msdn_html-куски\\z_windows development\\z_data acces and storage\\z_Local File Systems\\z_Directory Management\\About Directory Management (Windows)_files\\0d0542f5f45cf9fc7273abd11cf5c0ee.gif', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('D:\\Users\\feelus\\YandexDisk\\_programming\\_lang_cpp\\поиск файлов и msdn _html\\msdn_html-куски\\z_windows development\\z_data acces and storage\\z_Local File Systems\\z_Directory Management\\About Directory Management (Windows)_files\\8e010b67ceb176b577f5afa1ceb6f5ad.css', FileNotFou

[WinError 3] Системе не удается найти указанный путь: 'D:\\Users\\feelus\\YandexDisk\\_programming\\_lang_cpp\\поиск файлов и msdn _html\\_msdn_html\\z_windows development\\z_getting started\\z_learn to program  for windows in c++\\z_introduction to windows programming\\Prepare Your Development Environment (Windows)_files'
('D:\\Users\\feelus\\YandexDisk\\_programming\\_lang_cpp\\поиск файлов и msdn _html\\_msdn_html\\z_windows development\\z_getting started\\z_learn to program  for windows in c++\\z_introduction to windows programming\\Windows Coding Conventions (Windows)_files\\avatar(1).jpg', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('D:\\Users\\feelus\\YandexDisk\\_programming\\_lang_cpp\\поиск файлов и msdn _html\\_msdn_html\\z_windows development\\z_getting started\\z_learn to program  for windows in c++\\z_introduction to windows programming\\Windows Coding Conventions (Windows)_files\\avatar(2).jpg', FileNotFoundError(2, 'Системе не удается най

('D:\\Users\\feelus\\YandexDisk\\_programming\\_lang_cpp\\поиск файлов и msdn _html\\_msdn_html\\z_windows development\\z_getting started\\z_learn to program  for windows in c++\\z_Module 3. Windows Graphics\\Drawing with Direct2D (Windows)_files\\030c41d9079671d09a62d8e2c1db6973.gif', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('D:\\Users\\feelus\\YandexDisk\\_programming\\_lang_cpp\\поиск файлов и msdn _html\\_msdn_html\\z_windows development\\z_getting started\\z_learn to program  for windows in c++\\z_Module 3. Windows Graphics\\Drawing with Direct2D (Windows)_files\\0d0542f5f45cf9fc7273abd11cf5c0ee.gif', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('D:\\Users\\feelus\\YandexDisk\\_programming\\_lang_cpp\\поиск файлов и msdn _html\\_msdn_html\\z_windows development\\z_getting started\\z_learn to program  for windows in c++\\z_Module 3. Windows Graphics\\Drawing with Direct2D (Windows)_files\\8e010b67ceb176b577f5afa1ceb6

HTML(value='')

start writing
writed .files/last_snapshot 2019-11-21 12-32-21+0300.json
--- ITERATION ---
--- FINAL ITERATION ---
start writing
writed .files/patch 2019-11-21 11-55-30+0300 to 2019-11-21 12-32-21+0300.json


In [7]:
tree_stat(root)

(219441279589, 12138, 132489, 0)

# I

In [8]:
exclude_dirs = {r'I:\$RECYCLE.BIN'}
prefix = 'I:'
prefix_ = prefix+'\\'
last_diff_dir(prefix,prefix_,exclude_dirs,load_snapshot,dump_snapshot)

simple update
readed .files/last_snapshot 2019-11-21 12-05-39+0300.json
dict_keys(['errors', 'root'])


HTML(value='')

[WinError 5] Отказано в доступе: 'I:\\System Volume Information'
('I:\\_raw-backups\\backup\\AppData\\Local\\Packages\\29352GalleryImages.SexyAnimeCosplayGirlsDaily_8cearpz8v2nym\\LocalState\\MarkedUp\\Network\\Requests\\SessionStart\\MarkedUp.Infrastructure.Networking.NetworkServiceRequest\\635681119430186469_1_41cf8637e369493ab32f87e5c86b3b44', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('I:\\_raw-backups\\backup\\AppData\\Local\\Packages\\29352GalleryImages.SexyAnimeCosplayGirlsDaily_8cearpz8v2nym\\LocalState\\MarkedUp\\Network\\Requests\\SessionStart\\MarkedUp.Infrastructure.Networking.NetworkServiceRequest\\635681143364703554_3_4da8796b58c742bca8683412d2d47aa5', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('I:\\_raw-backups\\backup\\AppData\\Local\\Packages\\winstore_cw5n1h2txyewy\\LocalState\\Cache\\0\\0-DevApps-https∺∯∯next-services.apps.microsoft.com∯search∯6.3.9600-0∯776∯en-US_en-US.en.ru∯c∯RU∯cp∯10012737∯DevApps∯p

('I:\\_raw-backups\\backup\\AppData\\Local\\Packages\\winstore_cw5n1h2txyewy\\LocalState\\Cache\\0\\0-ProductTileExtendedByProductGuid-https∺∯∯next-services.apps.microsoft.com∯browse∯6.3.9600-0∯788∯en-US_en-US.en.ru∯c∯RU∯cp∯10012737∯Apps∯c08a0d72-28a1-465a-9e70-6a5b80b44d60.dat', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('I:\\_raw-backups\\backup\\AppData\\Local\\Packages\\winstore_cw5n1h2txyewy\\LocalState\\Cache\\0\\0-ProductTileExtendedByProductGuid-https∺∯∯next-services.apps.microsoft.com∯browse∯6.3.9600-0∯788∯en-US_en-US.en.ru∯c∯RU∯cp∯10012737∯Apps∯c0cb8c12-2fcf-4963-ba90-65cfbca5ebe2.dat', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('I:\\_raw-backups\\backup\\AppData\\Local\\Packages\\winstore_cw5n1h2txyewy\\LocalState\\Cache\\0\\0-ProductTileExtendedByProductGuid-https∺∯∯next-services.apps.microsoft.com∯browse∯6.3.9600-0∯788∯en-US_en-US.en.ru∯c∯RU∯cp∯10012737∯Apps∯c10aa66c-a9cd-4c28-8622-fc32c38c77a1.dat', FileNot

('I:\\_raw-backups\\C\\Users\\feelus\\AppData\\Local\\Microsoft\\Windows Store\\Cache\\0\\0-Namespace-https∺∯∯next-services.apps.microsoft.com∯browse∯6.3.9600-0∯776∯en-US.ru-RU∯c∯RU∯Namespace∯pc∯00000000-0000-0000-0000-000000000000∯00000000-0000-0000-0000-000000000000-.dat', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('I:\\_raw-backups\\C\\Users\\feelus\\AppData\\Local\\Packages\\29352GalleryImages.SexyAnimeCosplayGirlsDaily_8cearpz8v2nym\\LocalState\\MarkedUp\\Network\\Requests\\Default\\MarkedUp.Infrastructure.Networking.NetworkServiceRequest\\635681122173201901_2_512a2bc302564095b882f12107e1c6d1', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('I:\\_raw-backups\\C\\Users\\feelus\\AppData\\Local\\Packages\\29352GalleryImages.SexyAnimeCosplayGirlsDaily_8cearpz8v2nym\\LocalState\\MarkedUp\\Network\\Requests\\SessionStart\\MarkedUp.Infrastructure.Networking.NetworkServiceRequest\\635681119430186469_1_41cf8637e369493ab32f87e5c8

('I:\\_raw-backups\\C\\Users\\feelus\\AppData\\Local\\Packages\\winstore_cw5n1h2txyewy\\LocalState\\Cache\\0\\0-ProductTileExtendedByProductGuid-https∺∯∯next-services.apps.microsoft.com∯browse∯6.3.9600-0∯788∯en-US_en-US.en.ru∯c∯RU∯cp∯10012737∯Apps∯62208b9e-fbb1-4a46-b085-4fd8f834ecc5.dat', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('I:\\_raw-backups\\C\\Users\\feelus\\AppData\\Local\\Packages\\winstore_cw5n1h2txyewy\\LocalState\\Cache\\0\\0-ProductTileExtendedByProductGuid-https∺∯∯next-services.apps.microsoft.com∯browse∯6.3.9600-0∯788∯en-US_en-US.en.ru∯c∯RU∯cp∯10012737∯Apps∯62b6e527-e1ff-4526-aa35-3636a4a9fa39.dat', FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3))
('I:\\_raw-backups\\C\\Users\\feelus\\AppData\\Local\\Packages\\winstore_cw5n1h2txyewy\\LocalState\\Cache\\0\\0-ProductTileExtendedByProductGuid-https∺∯∯next-services.apps.microsoft.com∯browse∯6.3.9600-0∯788∯en-US_en-US.en.ru∯c∯RU∯cp∯10012737∯Apps∯6401827a-e608-4a67-a

[WinError 3] Системе не удается найти указанный путь: 'I:\\_wget_downloads\\D-wget-downloads\\khaydarkan.su\\arhivy_foto\\turizm\\Lysenko\\Ф\uf0d1%80агмен\uf0d1%82\uf0d1%8B о\uf0d1%82\uf0d1%87е\uf0d1%82а \uf0d0%92ладими\uf0d1%80а \uf0d0%9B\uf0d1%8B\uf0d1%81енко о водном п\uf0d1%83\uf0d1%82е\uf0d1%88е\uf0d1%81\uf0d1%82вии по \uf0d1%80екам \uf0d0%93а\uf0d1%83м\uf0d1%8B\uf0d1%88, \uf0d0%94жиали\uf0d1%81\uf0d1%83, Ходжаа\uf0d1%87кан, Со\uf0d1%85.files'


HTML(value='')

start writing
writed .files/last_snapshot 2019-11-21 12-33-23+0300.json
--- ITERATION ---
--- FINAL ITERATION ---
start writing
writed .files/patch 2019-11-21 12-05-39+0300 to 2019-11-21 12-33-23+0300.json


# Debug

In [9]:
raise BaseException()

BaseException: 

In [13]:
myjson_dumps({chr(0):0})

'{\n\t"\\u0000": 0\n}'

In [15]:
'\x00'

'\x00'

In [ ]:
import json

with open(r'.files/last_snapshot 2019-11-18 14-25-23+3000.bak','r') as file:
    tmp = tree_load(json.load(file))
    
del tmp['root']['Windows']['System32']['LogFiles']['HTTPERR']['httperr1.log']

with open(r'.files/last_snapshot 2019-11-18 14-25-23+3000.bak','w') as file:
    json.dump(tree_dump(tmp),file,indent='\t')


In [ ]:
from copy import *

len(old)
old_d = {}#deepcopy(old)
keys = [k for k in old.keys()]
for k in keys:#[2554:]:
    old_d[k]=deepcopy(old[k])
print(len(old_d))

len(new)
new_d = {}#deepcopy(old)
keys = [k for k in new.keys()]
for k in keys:#[3164:-1]:
    new_d[k]=deepcopy(new[k])
print(len(new_d))

#print(myjson_dumps(path_patch_dump({},old_d,new_d,{},{})))

r = hash_diff(old_d,new_d)


In [ ]:
old_root = myjson_load(r'D:\.files\111.json')
root = myjson_load(r'D:\.files\222.json')

#root = scan('D:\\',{r'D:\Users\feelus\YandexDisk',r'D:\$RECYCLE.BIN'})
#calc_hashes(root,old_root,'D:')

In [ ]:
oldroot_d = oldroot#{k:v for k,v in oldroot.items() if k!='__scan_errors__'}
root_d = root#{k:v for k,v in root.items() if k!='__scan_errors__'}
modified,old,new,strict_old,strict_new,touched = path_diff(oldroot_d,root_d)
assert set(old)&set(strict_old) == set()
assert set(new)&set(strict_new) == set()
moved,old_dirs,new_dirs,old,new = hash_diff(old,new)
assert set(old)&set(strict_old) == set()
assert set(new)&set(strict_new) == set()

newoldroot = hash_back_patch(root_d,modified,moved,old_dirs,new_dirs,
                      {**old,**strict_old},{**new,**strict_new},touched)
#first_diff(root_d,new_root_d)
assert oldroot_d==newoldroot



In [ ]:
if 0:
    old = {
        ('a','b'):[1,1,'1']
    }
    new = {
        ('b','c'):[2,2,'2']
    }
if 0:
    old = {
        ('a','b'):[1,1,'1']
    }
    new = {
        ('b','c'):[2,2,'1']
    }
if 0:
    old = {
        ('a','b'):[1,1,'1'],
        ('a','c'):[1,1,'1'],
    }
    new = {
        ('b','c'):[2,2,'1']
    }
if 1:
    old = {
        ('a','b'):[1,1,'1'],
        ('a','c'):[1,1,'1'],
    }
    new = {
        ('b','c'):[2,2,'1'],
        ('b','d'):[2,2,'1'],
    }
r = hash_diff(old,new)

print('moved',r[0])
print('old',r[1])
print('new',r[2])
print('old1',r[3])
print('new1',r[4])
